In [3]:
import pandas as pd
import csv

In [9]:
def read_file(f_name):
    data = []
    with open (f_name) as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) == 0:
                continue
            data.append(row)
    return data

In [55]:
def get_column_indices():
    sum_columns = list(range(2,12))
    per_capita_columns = list(range(12,23))

    return sum_columns, per_capita_columns

Loop through data files, the ones that I can to a data framed named by the state, then add the state column and bundle them all together. 

In [154]:
def build_state_index(data):
    state_index = {}
    state_start = 'Estimated crime in '
    for idx, row in enumerate(data):
        if state_start in row[0]:
            state = row[0].split(' ')[-1]
            if state not in state_index:
                state_index[idx] = state
#                 state_index[state]=idx
    return state_index

In [164]:
def get_header(row):
    header = row[:-1][:]
    sum_columns, per_capita_columns = get_column_indices()
    for sum_idx in sum_columns:
        h = row[sum_idx]
        new_header = 'Sum_{}'.format(h)
        header[sum_idx] = new_header
        
    for c in per_capita_columns[:-1]:

        h = header[c]

        new_header = 'Scaled_{}'.format(h)
        header[c] = new_header    
    
    return header
    
def clean_sample(sample, state):
    
    rt = []
    
    for row in sample:
        # skip all the empty rows and the headers 
        if len(row[0]) < 1:
            continue
        elif row[0] == 'Year':
            header = get_header(row)   
        else:
            rt.append(row)
            
    df = pd.DataFrame(rt, columns = header)
    df['State'] = [state for i in range(len(rt))]
    return df
    

def get_frames_per_state(state_index):

    l = list(state_index.keys())
    
    frames = []

    for i, s_idx in enumerate(l[:-1]):

        state = state_index[s_idx]
        
        s_idx_stop = l[i+1]
        
        sample = data[s_idx:s_idx_stop]

        cleaned = clean_sample(sample[1:], state)
        
        frames.append(cleaned)
        
    return pd.concat(frames)


data = read_file('../Data/CrimeStatebyState.csv')
state_index = build_state_index(data)
df = get_frames_per_state(state_index)

In [165]:
df.head()

,Year,Population,Sum_Violent crime total,Sum_Murder and nonnegligent Manslaughter,Sum_Legacy rape /1,Sum_Revised rape /2,Sum_Robbery,Sum_Aggravated assault,Sum_Property crime total,Sum_Burglary,...,Scaled_Murder and nonnegligent manslaughter rate,Scaled_Legacy rape rate /1,Scaled_Revised rape rate /2,Scaled_Robbery rate,Scaled_Aggravated assault rate,Scaled_Property crime rate,Scaled_Burglary rate,Scaled_Larceny-theft rate,Scaled_Motor vehicle theft rate,State
0,1960,3266740,6097,406,281,,898,4512,33823,11626,...,12.4,8.6,,27.5,138.1,1035.4,355.9,592.1,87.3,Alabama
1,1961,3302000,5564,427,252,,630,4255,32541,11205,...,12.9,7.6,,19.1,128.9,985.5,339.3,569.4,76.8,Alabama
2,1962,3358000,5283,316,218,,754,3995,35829,11722,...,9.4,6.5,,22.5,119.0,1067.0,349.1,634.5,83.4,Alabama
3,1963,3347000,6115,340,192,,828,4755,38521,12614,...,10.2,5.7,,24.7,142.1,1150.9,376.9,683.4,90.6,Alabama
4,1964,3407000,7260,316,397,,992,5555,46290,15898,...,9.3,11.7,,29.1,163.0,1358.7,466.6,784.1,108.0,Alabama


In [166]:
df.to_csv("../Data/cleaned_crime.csv", index=False)